In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download vipoooool/new-plant-diseases-dataset

In [ ]:
!unzip new*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

In [ ]:
len(os.listdir("/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train"))

In [ ]:
train_datagen = ImageDataGenerator(zoom_range=0.5, shear_range=0.3, horizontal_flip=True, preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train = train_datagen.flow_from_directory(directory="/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train", target_size=(256,256), batch_size=32)

val = val_datagen.flow_from_directory(directory="/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid", target_size=(256,256), batch_size=32)

MODEL BUILDING

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
import keras
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

In [ ]:
base_model = VGG19(input_shape=(256, 256, 3), include_top=False)

In [ ]:
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
X = Flatten()(base_model.output)

X = Dense(units=38, activation='softmax')(X)

model = Model(base_model.input, X)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
#early stopping and model checkpoint

from keras.callbacks import ModelCheckpoint, EarlyStopping

#early stopping
es = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.01, patience = 3, verbose = 1)

#model checkpoint
mc = ModelCheckpoint(filepath = "best_model.h5", min_delta = 0.01, patience = 3, verbose = 1, save_best_onlu = True)

cb = [es, mc]

In [ ]:
his = model.fit_generator(train, steps_per_epoch=16, epochs = 50, verbose = 1, callbacks = cb, validation_data=val, validation_steps=16)

In [ ]:
h = his.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c = "red")
plt.title("acc vs val_acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c = "red")
plt.title("loss vs val_loss")
plt.show()

In [ ]:
#load best model

from keras.models import load_model

model = load_model("/content/best_model.h5")

In [ ]:
acc = model.evaluate_generator(val)[1]

print(f"The accuracy of your model is = {acc*100} %")

In [ ]:
ref = dict(zip( list(train.class_indices.values()), list(train.class_indices.keys())) )

In [ ]:
def prediction(path):

  img = load_img(path, target_size = (256,256))

  i = img_to_array(img)

  im = preprocess_input(i)

  img = np.expand_dims(im, axis = 0)

  pred = np.argmax(model.predict(img))

  print(f" the image belongs to { ref[pred] } ")

In [ ]:
path = "/content/test/test/AppleScab1.JPG"

prediction(path)